# Proyecto 1 - Explorando los Algoritmos de Aprendizaje Supervisado
## Juan Ignacio Navarro
## Jose David Sánchez
## Steven Badilla

### Implementación de los algoritmos de aprendizaje supervisado
* Regresión Logística

In [ ]:
# Algoritmo de regresión logística

* Árboles de decisión

In [1]:
# Algoritmo de árboles de decision

* KNN

In [2]:
# Algoritmo de KNN

### Prueba de los algoritmos con set de datos

* Set de datos de Red Wine Quality

In [3]:
# feature engineering 
# prueba con los diferentes algoritmos
# comparación de los resultados

* Set de datos de notas del curso de Arquitectura de Computadores I

In [ ]:
# feature engineering 
# prueba con los diferentes algoritmos
# comparación de los resultados

* Set de datos a elegir

In [ ]:
# feature engineering 
# prueba con los diferentes algoritmos
# comparación de los resultados

### Puntos extra: modelo que precide la nota del estudiante (utilizando el set de datos de Arquitectura de Computadores I)

La predicción está basada en las notas del proyecto 1, proyecto 2, examen 1 y taller 1.